# Введение в нейронные сети

## Урок 5. Рекуррентные нейронные сети

1. Попробуйте починить сеть по словам.

2. Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший текст из получившихся и опишите предпринятые для его получения действия. Можно использовать текст другого произведения

In [466]:
import re

import numpy as np

import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Input, SimpleRNN, Dense, GRU, LSTM, Bidirectional
from keras.utils import to_categorical

In [467]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
text

'Вы — лучший ответ на проблемы, которые возникли в понедельник.\nДумайте позитивно и верьте в свою способность достигать отличных результатов.\nЕсли вы смогли в понедельник подняться с постели, значит вы супер герой.'

In [468]:
maxWordsCount = 1000
token = Tokenizer(num_words=maxWordsCount,
                  filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                  lower=True,
                  split=' ',
                  char_level=False)

In [469]:
token.fit_on_texts([text])

In [470]:
token.index_word

{1: 'вы',
 2: 'в',
 3: 'понедельник',
 4: 'лучший',
 5: 'ответ',
 6: 'на',
 7: 'проблемы',
 8: 'которые',
 9: 'возникли',
 10: 'думайте',
 11: 'позитивно',
 12: 'и',
 13: 'верьте',
 14: 'свою',
 15: 'способность',
 16: 'достигать',
 17: 'отличных',
 18: 'результатов',
 19: 'если',
 20: 'смогли',
 21: 'подняться',
 22: 'с',
 23: 'постели',
 24: 'значит',
 25: 'супер',
 26: 'герой'}

In [471]:
token.word_index

{'вы': 1,
 'в': 2,
 'понедельник': 3,
 'лучший': 4,
 'ответ': 5,
 'на': 6,
 'проблемы': 7,
 'которые': 8,
 'возникли': 9,
 'думайте': 10,
 'позитивно': 11,
 'и': 12,
 'верьте': 13,
 'свою': 14,
 'способность': 15,
 'достигать': 16,
 'отличных': 17,
 'результатов': 18,
 'если': 19,
 'смогли': 20,
 'подняться': 21,
 'с': 22,
 'постели': 23,
 'значит': 24,
 'супер': 25,
 'герой': 26}

In [472]:
dist = list(token.word_counts.items())
dist

[('вы', 3),
 ('лучший', 1),
 ('ответ', 1),
 ('на', 1),
 ('проблемы', 1),
 ('которые', 1),
 ('возникли', 1),
 ('в', 3),
 ('понедельник', 2),
 ('думайте', 1),
 ('позитивно', 1),
 ('и', 1),
 ('верьте', 1),
 ('свою', 1),
 ('способность', 1),
 ('достигать', 1),
 ('отличных', 1),
 ('результатов', 1),
 ('если', 1),
 ('смогли', 1),
 ('подняться', 1),
 ('с', 1),
 ('постели', 1),
 ('значит', 1),
 ('супер', 1),
 ('герой', 1)]

In [473]:
data = token.texts_to_sequences([text])

In [474]:
data_cat = to_categorical(data[0], num_classes=maxWordsCount)
data_cat.shape

(31, 1000)

In [475]:
inp_words = 3
n = data_cat.shape[0]-inp_words

In [476]:
X = np.array([data_cat[i:i+inp_words, :] for i in range(n)])
Y = data_cat[inp_words:]

In [477]:

model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
# model.add(SimpleRNN(128, activation='tanh'))

# model.add(GRU(128, activation='tanh'))
# model.add(LSTM(128, activation='tanh'))
model.add(Bidirectional(SimpleRNN(128, activation='tanh')))

model.add(Dense(maxWordsCount, activation='softmax'))



In [478]:
model.summary()

Model: "sequential_30"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_3 (Bidirectional) │ (None, 256)            │       289,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1000)           │       257,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 546,024 (2.08 MB)

 Trainable params: 546,024 (2.08 MB)

 Non-trainable params: 0 (0.00 B)

In [479]:
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')


In [480]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 6.9141
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0000e+00 - loss: 6.8664
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.1071 - loss: 6.8185
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3214 - loss: 6.7701
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6429 - loss: 6.7207
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9286 - loss: 6.6696
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 6.6163
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 6.5602
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 6.5007
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 1.0000 - loss: 6.4372
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 1.0000 - loss: 6.3688
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 6.2946
Epoch 

In [486]:
def buildPhrase(text, str_len = 22):
    out_str = text
    in_data = token.texts_to_sequences([text])[0]
    for i in range(str_len):
        x = to_categorical(in_data[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
        inp = x.reshape(1, inp_words, maxWordsCount)
        pred = model.predict(inp) # предсказываем OHE четвертого символа
        indx = pred.argmax(axis=1)[0]
        in_data.append(indx)
        out_str += " " + token.index_word[indx] # дописываем строку
    return out_str

In [487]:
buildPhrase("вы в понедельник")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


'вы в понедельник думайте позитивно и верьте в свою способность достигать отличных результатов если вы смогли в понедельник подняться с постели значит вы супер герой'

### Описание предпринятых действий:  
- Подготовка данных: Сначала читаем текстовый файл train_data.txt, удаляем невидимые символы и преобразуем текст в последовательности чисел с помощью Tokenizer из Keras. Этот процесс включает в себя:
- Удаление специальных символов и преобразование текста в нижний регистр.
- Ограничение словаря до 1000 наиболее часто встречающихся слов.
- Преобразование текста в последовательности чисел, где каждое число представляет индекс слова в словаре.
- Преобразование данных: Вы преобразуете эти последовательности в one-hot кодировку, чтобы модель могла работать с категориальными данными. Затем вы разделяете данные на входные (X) и выходные (Y) части, используя окно размером 3 слова для входных данных и оставляя остаток для выходных данных.
Создание модели: создаем модель с использованием Sequential API Keras, добавляя в неё слои. Модель начинается с входного слоя, затем использует Bidirectional слой с SimpleRNN для обработки последовательностей, и заканчивается плоским слоем с функцией активации softmax для предсказания следующего слова.
- Обучение модели: обучаем модель на подготовленных данных, используя оптимизатор Adam и функцию потерь categorical_crossentropy.
- Генерация текста: После обучения модель используется для генерации новых фраз на основе входного текста. Функция buildPhrase принимает начальный текст и длину генерируемой фразы, преобразует текст в one-hot кодировку, использует модель для предсказания следующего слова и добавляет его к исходному тексту. Этот процесс повторяется до тех пор, пока не будет сгенерирована фраза заданной длины.